In [3]:
import pandas as pd
import numpy as np

store = pd.HDFStore('/Users/jiaqizhang/Documents/innovation_day/stock/data/base_store.h5')
base = store['base'] 


In [5]:
base.columns

dev = base[base.transaction_date < pd.to_datetime('2015-01-01')]

In [ ]:
store.close()

In [7]:
dev.columns

Index([u'code', u'close', u'turn', u'mkt_cap', u'open', u'vol', u'adjfactor',
       u'amount', u'swing', u'high', u'low', u'transaction_date', u'ipo_date',
       u'weekday', u'out_date', u'close_7d', u'open_7d', u'closed_diff',
       u'issue_date', u'opprofit', u'non_oper_rev', u'non_oper_exp',
       u'tot_profit', u'tax', u'net_profit_is', u'minority_int_inc',
       u'np_belongto_parcomsh', u'net_cash_flows_oper_act',
       u'stot_cash_outflows_oper_act', u'stot_cash_outflows_inv_act',
       u'net_cash_flows_inv_act', u'stot_cash_outflows_fnc_act',
       u'net_cash_flows_fnc_act', u'monetary_cap', u'tradable_fin_assets',
       u'notes_rcv', u'acct_rcv', u'oth_rcv', u'prepay', u'dvd_rcv',
       u'int_rcv', u'inventories', u'tot_cur_assets',
       u'fin_assets_avail_for_sale', u'held_to_mty_invest',
       u'invest_real_estate', u'long_term_eqy_invest', u'long_term_rec',
       u'fix_assets', u'intang_assets', u'r_and_d_costs', u'goodwill',
       u'long_term_deferred_exp', u

In [9]:
dev_3 = dev[dev.weekday == 3]
dev_3

,code,close,turn,mkt_cap,open,vol,adjfactor,amount,swing,high,...,lt_borrow,bonds_payable,tot_liab,eqy_belongto_parcomsh,cap_rsrv,surplus_rsrv,undistributed_profit,other_equity_instruments_PRE,tot_liab_shrhldr_eqy,days_since_ipo
2,000001.sz,5.42,0.67,5.509807e+10,5.46,20653127.0,35.91,3.271489e+08,2.39,5.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7218.0
7,000001.sz,5.60,0.63,5.694512e+10,5.65,19481115.0,35.91,3.196641e+08,1.46,5.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7225.0
12,000001.sz,5.11,0.93,5.199641e+10,5.31,28757900.0,35.91,4.355242e+08,3.62,5.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7232.0
17,000001.sz,5.25,0.82,5.342526e+10,5.19,25551127.0,35.91,3.893199e+08,2.62,5.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7239.0
22,000001.sz,5.30,0.44,5.384346e+10,5.23,13682225.0,35.91,2.101390e+08,1.77,5.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7253.0
27,000001.sz,5.48,0.82,5.569052e+10,5.47,25381007.0,35.91,4.058368e+08,2.08,5.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7260.0
32,000001.sz,5.33,0.65,5.422681e+10,5.34,20230781.0,35.91,3.134731e+08,1.22,5.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7267.0
37,000001.sz,5.62,3.16,5.715423e+10,5.55,98048937.0,35.91,1.616932e+09,3.65,5.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7274.0
42,000001.sz,5.57,0.99,5.663147e+10,5.69,30849383.0,35.91,5.041658e+08,2.28,5.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7281.0
47,000001.sz,5.38,0.67,5.474957e+10,5.42,20726243.0,35.91,3.270397e+08,1.38,5.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7288.0


In [13]:
meta_variables = ['code', 'transaction_date', 'ipo_date', 'out_date', 'close_7d', 'open_7d', 'closed_diff', 'issue_date']
all_vars = dev_3.columns
key_vars = ['code', 'transaction_date']
target_var = ['closed_diff']

In [14]:
candidate_vars = [var for var in all_vars if var not in meta_variables]
candidate_vars

['close',
 'turn',
 'mkt_cap',
 'open',
 'vol',
 'adjfactor',
 'amount',
 'swing',
 'high',
 'low',
 'weekday',
 'opprofit',
 'non_oper_rev',
 'non_oper_exp',
 'tot_profit',
 'tax',
 'net_profit_is',
 'minority_int_inc',
 'np_belongto_parcomsh',
 'net_cash_flows_oper_act',
 'stot_cash_outflows_oper_act',
 'stot_cash_outflows_inv_act',
 'net_cash_flows_inv_act',
 'stot_cash_outflows_fnc_act',
 'net_cash_flows_fnc_act',
 'monetary_cap',
 'tradable_fin_assets',
 'notes_rcv',
 'acct_rcv',
 'oth_rcv',
 'prepay',
 'dvd_rcv',
 'int_rcv',
 'inventories',
 'tot_cur_assets',
 'fin_assets_avail_for_sale',
 'held_to_mty_invest',
 'invest_real_estate',
 'long_term_eqy_invest',
 'long_term_rec',
 'fix_assets',
 'intang_assets',
 'r_and_d_costs',
 'goodwill',
 'long_term_deferred_exp',
 'deferred_tax_assets',
 'tot_non_cur_assets',
 'tot_assets',
 'st_borrow',
 'notes_payable',
 'acct_payable',
 'adv_from_cust',
 'empl_ben_payable',
 'taxes_surcharges_payable',
 'tot_acct_payable',
 'int_payable',
 '

In [21]:
dev_3[candidate_vars].dtypes[51:]

adv_from_cust                   float64
empl_ben_payable                float64
taxes_surcharges_payable        float64
tot_acct_payable                float64
int_payable                     float64
dvd_payable                     float64
oth_payable                     float64
tot_cur_liab                    float64
lt_borrow                       float64
bonds_payable                   float64
tot_liab                        float64
eqy_belongto_parcomsh           float64
cap_rsrv                        float64
surplus_rsrv                    float64
undistributed_profit            float64
other_equity_instruments_PRE    float64
tot_liab_shrhldr_eqy            float64
days_since_ipo                  float64
dtype: object

In [22]:
dev_3_train = dev_3[dev_3.transaction_date > pd.to_datetime('2013-01-01') ]
dev_3_test = dev_3[dev_3.transaction_date <= pd.to_datetime('2013-01-01') ]
oot = base[base.transaction_date >= pd.to_datetime('2015-01-01')]
oot_3 = oot[oot.weekday == 3]


In [23]:
store['dev_3_train'] = dev_3_train
store['dev_3_test'] = dev_3_test
store['oot_3'] = oot_3

In [24]:
dev_3_train.to_csv('/Users/jiaqizhang/Documents/innovation_day/stock/data/dev_3_train.csv')
dev_3_test.to_csv('/Users/jiaqizhang/Documents/innovation_day/stock/data/dev_3_test.csv')
oot_3.to_csv('/Users/jiaqizhang/Documents/innovation_day/stock/data/oot_3.csv')


In [37]:
dev_3_train = dev_3_train[dev_3_train.closed_diff.notnull()]
dev_3_train = dev_3_train[dev_3_train.issue_date.notnull()]

train_y = dev_3_train[target_var]

In [39]:
train_x = dev_3_train[candidate_vars]

from sklearn.preprocessing import Imputer
x = Imputer().fit_transform(train_x)





In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)

X = scaler.transform(x)


In [ ]:
train

In [41]:
from sklearn import linear_model
clf = linear_model.SGDRegressor()

clf.fit(X, train_y)

/Users/jiaqizhang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)